In [1]:
import pathpyG as pp
pp.config['torch']['device'] = 'cpu'

In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import scatter

PahtpyG takes as input various types of data: 
- Paths: DAG class
- DAGs: DAG class 
- Edgelists: DAG e network
- Time stamped interactions (events)


These data can then be represented with different types of models: 
- Network: Graph class
- Higher-Order Network (as layers of a mon)
- Multi-Order network
- Temporal Network

The focus of the package is teh ability to represent, model, use, memory in interactions within statistical, machine learning, and deep learning methods. 

**Passing walks to DAG class**

Walks can be passed in different ways. 
The most intuitive ways is to pass them as tuples (iterables?)  to a DAGData object.
This approach requires a mapping from string ids to node indices (these mappings are handled by IndexMap). 
Such a mapping can be conveniently obtained intitializing a network object.
The network object represent the topological backbone traversed by the walk dynamics.

In [3]:
g = pp.Graph.from_edge_list([('a', 'b'), ('b', 'c'), ('a', 'c')])
dags = pp.DAGData(mapping = g.mapping)

dags.append_walk(('a', 'b', 'c', 'b'), weight=1.0)
dags.append_walk(('a', 'c'), weight = 2.0)
print(dags)

DAGData with 2 dags with total weight 3.0


otehrwise, we can independently initilize an EdgeIndex object

In [4]:
dags = pp.DAGData(pp.IndexMap(list("abc")))
dags.append_walk(('a', 'b', 'c', 'b'), weight=1.0)
dags.append_walk(('a', 'c'), weight = 2.0)
print(dags)

DAGData with 2 dags with total weight 3.0


Finally, we can pass walks as edge indices without specifying a mapping. 
Notice that for the node indices to represent a valid walk, all subsequent edges must be adjecent.
In the edge_index format, this means that i-th element of the target indices must be equal to i+1 element of the source indices. 
Intuitively, this represents the fact that the node receives the path as a target and then propagates the path as a source. 

In [5]:
dags = pp.DAGData()
dags.append_dag(torch.tensor([[1,2,3,4],[2,3,4,5]]), weight=1)
dags.append_dag(torch.tensor([[3,4,5,6],[4,5,6,7]]), weight=2)
print(dags)

DAGData with 2 dags with total weight 3


**Passing DAGs to DAG class**

Naturally, we can also pass DAGs to the DAG class. 
Both with and without IndexMap, the operation is now perfomed using the append_dag method. 
In a dag, we are no longer constrained to pass edge indices where the i-th element for the target is equal to the i+1 elemen fof the source.  
This is a consequence of the fact that DAGs have bifurcations while walks, by definition, cannot. 
The edge_index of a DAG represents source target intearctions in the dag. 
[[0,0][1,2]] represents the root node (at t_0) interacting with three other nodes at times t_1, t_2. 


Notice that the current implementation of the DAG class cannot represent DAGs with the same node appearing at different times. 
For example, in [[0,0,1],[1,2,2]] we are saying that the node with index 1 hits the same 2 that was hit by 0 (i.e., 2 as indegree 2). 
This representation, however, does not allow us to say that 2 hits 1 at a later time (leading to two copies of the node, both with indegree one)

In [6]:
dags = pp.DAGData()
dags.append_dag(torch.tensor([[0,0,1],[1,2,2]]))

In [7]:
dags = pp.DAGData(pp.IndexMap(list("abc")))
dags.append_dag(torch.tensor([[0,0,1],[1,2,2]]))
print(dags.dags)

[Data(edge_index=[2, 3], node_sequence=[3, 1], num_nodes=3, edge_weight=[3])]


**Passing Walks and DAGs**
Finally, we can pass both walks and dags at the same time

CURRELTY BUGGY (bug appears when training a multi order network)

In [8]:
# Example with mix of walks or dags
dags = pp.DAGData(mapping = g.mapping)

dags.append_dag(torch.tensor([[0,0,1],[1,2,2]]), weight=2)
dags.append_walk(('a', 'b', 'c'))
print(dags)

m = pp.MultiOrderModel.from_DAGs(dags, max_order=2)

DAGData with 2 dags with total weight 3.0


### Temporal network

In [9]:
torch.repeat_interleave(torch.tensor([2,3,4]))

tensor([0, 0, 1, 1, 1, 2, 2, 2, 2])

In [10]:
import torch_geometric
# import torch_geometric.utils
we = torch_geometric.utils.cumsum(torch.tensor([1,1,1,1,1,1]), dim = 0)[:-1]
torch.repeat_interleave(we)

tensor([1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5])

### Multi-Order model
 

In [11]:
def compute_weighted_outdegrees(graph):
    weighted_outdegree = scatter(graph.data.edge_weight, graph.data.edge_index[0], dim=0, dim_size=graph.data.num_nodes, reduce='sum')
    return weighted_outdegree

def compute_transition_probabilities(graph):
    weighted_outdegree = compute_weighted_outdegrees(graph)
    source_ids = graph.data.edge_index[0]
    return graph.data.edge_weight/ weighted_outdegree[source_ids]


In [12]:
dag_data = pp.DAGData(pp.IndexMap(list("01234")))

# dag_data.append_dag(torch.tensor([[0,2],[2,3]]), weight=20)
# dag_data.append_dag(torch.tensor([[1,2],[2,4]]), weight=20)
# print(dag_data)

dag_data.append_walk(list("0230230"), weight=30)
dag_data.append_walk(list("1241241"), weight=70)
dag_data.append_walk(list("0230241"), weight=1)

m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=7)

hon_1 = m.layers[1]
hon_2 = m.layers[2]
hon_3 = m.layers[3]
hon_4 = m.layers[4]
hon_5 = m.layers[5]
hon_6 = m.layers[5]
hon_7 = m.layers[7]
print(hon_1.data.edge_weight)
print(hon_2.data.edge_weight)

t_1 = compute_transition_probabilities(hon_1)
t_2 = compute_transition_probabilities(hon_2)

print(t_1)
print(t_2)

tensor([ 62, 140,  61, 141,  61, 141])
tensor([ 61,   1, 140,  61, 141,  31,  70])
tensor([1.0000, 1.0000, 0.3020, 0.6980, 1.0000, 1.0000])
tensor([0.9839, 0.0161, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [13]:
def generate_general_bipartite_edge_index(g_source, g_target) -> torch.Tensor:
    """Generate edge_index for bipartite graph connecting nodes of higher-order graphs with arbitrary (but different) orders."""
    order_source = g_source.data.node_sequence[0].shape[0]
    order_target = g_target.data.node_sequence[0].shape[0]
    assert order_source!= order_target, "Source and target must have different orders to generate bipartite indices"
    node_sequence_source = g_source.data.node_sequence
    node_sequence_target = g_target.data.node_sequence
    d_order = max(order_source, order_target) - min(order_source, order_target)
    if order_source>order_target:
        mask = torch.all(node_sequence_target[:,None] == node_sequence_source[:, d_order:], dim=-1).T
        bip_tensor = torch.nonzero(mask, as_tuple=False).T
    else:
        mask = torch.all(node_sequence_source[:, None] == node_sequence_target[:, :-d_order], dim=-1)
        bip_tensor = torch.nonzero(mask, as_tuple=False).T
    return bip_tensor
    

In [14]:
# import numpy as np
# class Lh_conv(torch_geometric.nn.MessagePassing):
#     def __init__(self):
#         super().__init__(aggr="sum", flow="source_to_target", node_dim = -1)
#     def forward(self, x_source, source_to_target_edge_index, transition_probability):
        
#         N = len(source_to_target_edge_index[0].unique())
#         M = len(source_to_target_edge_index[1].unique())

#         return self.propagate(
#                     source_to_target_edge_index,
#                     size = (N,M),
#                     x = (x_source, None),
#                     transition_probability = transition_probability
#                     )

#     def message(self, x_j, transition_probability):
#         return x_j + np.log(transition_probability)

In [15]:
# lh_conv = Lh_conv()
# x_source = torch.tensor([np.log(1)]) # lh of path until now # here 1 cause considering the example of a path set{(0,0),(0,1)}, hence only have a deterministic 0th transition *->0
# # x_target = None # these are gonna be genetaed
# source_to_target_edge_index = torch.tensor([[0,0],[0,1]])
# transition_probability = torch.tensor([.3,.7])
# vec_log_lh = lh_conv(x_source, source_to_target_edge_index, transition_probability)
# print(vec_log_lh.sum())

Gettin zeroth order

In [16]:
dag_data.dags

[Data(edge_index=[2, 6], node_sequence=[7, 1], num_nodes=7, edge_weight=[6]),
 Data(edge_index=[2, 6], node_sequence=[7, 1], num_nodes=7, edge_weight=[6]),
 Data(edge_index=[2, 6], node_sequence=[7, 1], num_nodes=7, edge_weight=[6])]

In [17]:
from torch_geometric.loader import DataLoader
dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(dag_data.dags)))).to(pp.config["torch"]["device"])
edge_index = dag_graph.edge_index
node_sequence = dag_graph.node_sequence

In [18]:
hon_4.mapping.id_to_idx

{('0', '2', '3', '0'): 0,
 ('0', '2', '4', '1'): 1,
 ('1', '2', '4', '1'): 2,
 ('2', '3', '0', '2'): 3,
 ('2', '4', '1', '2'): 4,
 ('3', '0', '2', '3'): 5,
 ('3', '0', '2', '4'): 6,
 ('4', '1', '2', '4'): 7}

In [19]:
unique_nodes, counts = torch.unique(dag_graph.node_sequence, return_counts=True)

node_emission_probabilities = counts/counts.sum()
node_emission_probabilities

tensor([0.2381, 0.1905, 0.2857, 0.1429, 0.1429])

In [20]:
import torch
# !!!!!!!!!!!!!!!!!!
# Define the Cantor pairing function to work with tensors
# https://en.wikipedia.org/wiki/Pairing_function
def cantor_pairing(x, y):
    return (x + y) * (x + y + 1) // 2 + y

# Define the function to encode a list into a single integer
def encode_tensor(lst):
    if lst.size(1) == 0:
        return torch.tensor(0)
    else:
        return cantor_pairing(lst[:, 0], encode_tensor(lst[:, 1:]))

tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Apply the encode_list function row-wise
result = encode_tensor(tensor)

print(result)

tensor([    988,   71248, 1101862])


Now we used this encoding also on node sequences, and then map supaths to ho-nodes ids

In [21]:
dict_cantor_to_honode_ixs_mapping = dict()
for order, hon in m.layers.items():
    # print(order)
    # Creating cantor ixs for nodes in node sequence of hon
    cantor_ids  = encode_tensor(hon.data.node_sequence)

    # mapping cantor ids to node ids
    cantor_to_node_ixs_mapping = dict(zip(
        cantor_ids.tolist(),torch.arange(cantor_ids.shape[0]).tolist()
        ))
    # adding mapping for sink node
    # cantor_to_node_ixs_mapping[sink_padding] = cantor_ids.shape[0]
    dict_cantor_to_honode_ixs_mapping[order] = cantor_to_node_ixs_mapping
dict_cantor_to_honode_ixs_mapping

{1: {0: 0, 1: 1, 3: 2, 6: 3, 10: 4},
 2: {9: 0, 13: 1, 42: 2, 88: 3, 6: 4, 16: 5},
 3: {945: 0, 4004: 1, 4093: 2, 42: 3, 187: 4, 87: 5, 166: 6},
 4: {945: 0,
  17765: 1,
  17953: 2,
  4092: 3,
  14362: 4,
  450771: 5,
  8034032: 6,
  8398846: 7},
 5: {8378370: 0,
  103169428: 1,
  101598824922: 2,
  32272863207627: 3,
  35270336461822: 4,
  450771: 5,
  157877561: 6,
  161253856: 7},
 6: {-3927727720672062055: 0,
  -1544122977309695571: 1,
  -3095025234462892546: 2,
  101598824922: 3,
  12462662686225827: 4,
  13001403601822867: 5},
 7: {-3927727720672062055: 0, 2441141984275317593: 1, 930405143996658085: 2}}

**Need to apply the encoding before the padding.**
The issue is that we don t know how the tensor encoding will work when applied to the 'sink_padding' (so, we need to apply the encoding before the padding!)

In [22]:
from torch.nn.utils import rnn as rnn_utils


# # # https://suzyahyah.github.io/pytorch/2019/07/01/DataLoader-Pad-Pack-Sequence.html
# def pad_collate(batch):
#     # X =
#     X_pad = rnn_utils.pad_sequence(batch, batch_first=True, padding_value = -1) # torch.as_tensor(target_train).view(-1, n_features).float()
#     return X_pad



# class CustomDataset(Dataset):
#     def __init__(self, dag_data):
#         self.list_node_seq_paths = [encode_tensor(dag.node_sequence.T) for dag in dag_data.dags]
        
#     def __getitem__(self, index):
#         return self.list_node_seq_paths[index]

#     def __len__(self):
#         return len(self.list_node_seq_paths)

from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset): # right now doesn t need to inherit from dataset (?)
    def __init__(self, dag_data,
                # dict_cantor_to_honode_ixs_mapping, 
                max_order):
        self.max_order = max_order
        self.walks_by_length = {}
        self.walk_counts_by_length = {}
        # self.dict_cantor_to_honode_ixs_mapping = dict_cantor_to_honode_ixs_mapping
        for dag in dag_data.dags:
            node_seq_path = dag.node_sequence.T[0]
            l = len(node_seq_path)
            if l not in self.walks_by_length:
                self.walks_by_length[l] = []
                self.walk_counts_by_length[l] = []
            self.walks_by_length[l].append(node_seq_path)
            self.walk_counts_by_length[l].append(int(dag.edge_weight.unique()))
        self.total_sequences = sum(len(seq_list) for seq_list in self.walks_by_length.values())
        self.walk_tensors_by_length = {l:torch.stack(walks, dim = 0) for l,walks in self.walks_by_length.items()}
        self.cantor_encoded_walks_by_length = {l:self.cantor_encode(l) for l in self.walk_tensors_by_length}
        self.bipartite_encoded_walks_by_length = {l:self.bipartite_encode(l) for l in self.walk_tensors_by_length}

    def cantor_encode(self, walk_length):
        list_cantor_node_ixs_tensors = []
        for i in range(1,walk_length+1): # print("should probably start from zero")
            # need only one, not soirce target... 
            hon_ixs_tensor = self.walk_tensors_by_length[walk_length][:,max(0,i-self.max_order):i]
            list_cantor_node_ixs_tensors.append(encode_tensor(hon_ixs_tensor))
        return torch.stack(list_cantor_node_ixs_tensors, dim=1)


    def bipartite_encode(self, walk_length):
        list_cantor_node_ixs_tensors = []
        for i in range(1,walk_length+1): # print("should probably start from zero")
            # need only one, not soirce target... 
            hon_ixs_tensor = self.walk_tensors_by_length[walk_length][:,max(0,i-self.max_order):i]
            list_cantor_node_ixs_tensors.append(
                encode_tensor(hon_ixs_tensor).apply_(
                    dict_cantor_to_honode_ixs_mapping[min(i,self.max_order)].get))
                
                
        return torch.stack(list_cantor_node_ixs_tensors, dim=1)

    def __getitem__(self, l ,index):
        return self.bipartite_encoded_walks_by_length[l][index]

    def __len__(self):
        return self.total_sequences




cd_data = CustomDataset(dag_data,max_order=2)

# data_loader = DataLoader(cd_data, batch_size=4, shuffle=False, collate_fn=pad_collate)


# from torch.utils.data import DataLoader, Dataset, TensorDataset


Still need:
- account for number of observations of said path
- deal with computing lh (passing those bip indices through message passing)

In [23]:
l =7
source_to_target_from_walks = cd_data.bipartite_encoded_walks_by_length[l]
path_counts = cd_data.walk_counts_by_length[l]

In [24]:
import numpy as np
class Lh_conv(torch_geometric.nn.MessagePassing):
    def __init__(self):
        super().__init__(aggr="sum", flow="source_to_target", node_dim = -1)
    def forward(self, x_source, source_to_target_edge_index, transition_probability,N,M):
        
        # N = len(source_to_target_edge_index[0].unique())
        # M = len(source_to_target_edge_index[1].unique())

        return self.propagate(
                    source_to_target_edge_index,
                    size = (N,M),
                    x = (x_source, None),
                    transition_probability = transition_probability
                    )

    def message(self, x_j, transition_probability):
        return x_j + np.log(transition_probability)

In [25]:

n_nodes = len(unique_nodes)
lh_conv = Lh_conv()
x_source = torch.zeros(1)# ((n_nodes,1)) 
# x_target = None # these are gonna be genetaed
print("0-th")
print(node_emission_probabilities)
M = n_nodes
N = 1
source_to_target_edge_index_zeroth = torch.stack([
    torch.zeros_like(source_to_target_from_walks[:,0]),
    source_to_target_from_walks[:,0]]
    )
vec_log_lh = lh_conv(
    x_source,
    source_to_target_edge_index_zeroth, 
    torch.pow(node_emission_probabilities[source_to_target_edge_index_zeroth[1]], torch.tensor(path_counts)),
    N,
    M
    )
print(vec_log_lh)
# multiply for the number of time the path has been observed? 

# HERE is max_order
# enough to just do it for all paths lenghts? 
# TODO: need to include path counts
for i in range(0,l-1):
    print(min(i+1,cd_data.max_order))
    T = compute_transition_probabilities(m.layers[min(i+2,cd_data.max_order)])
    M = m.layers[min(i+2,cd_data.max_order)].data.num_nodes
    N = m.layers[min(i+1,cd_data.max_order)].data.num_nodes
    source_to_target_edge_index = source_to_target_from_walks[:,i:i+2].T.squeeze() # TERRIBLE INDEXING HERE
    print("source to target",source_to_target_edge_index.shape)
    print("transition probabilities",T.shape)
    vec_log_lh = lh_conv(
        vec_log_lh, 
        source_to_target_edge_index, 
        torch.pow(T[source_to_target_edge_index[1]], torch.tensor(path_counts)),
        N,
        M
        ) # weights given by target (e.g. given by edge weight n node to edge incdence) (CHECK!!!)
    print("ok")
    print(vec_log_lh.shape)




0-th
tensor([0.2381, 0.1905, 0.2857, 0.1429, 0.1429])
tensor([-44.4876,     -inf,   0.0000,   0.0000,   0.0000])
1
source to target torch.Size([2, 3])
transition probabilities torch.Size([7])
ok
torch.Size([6])
2
source to target torch.Size([2, 3])
transition probabilities torch.Size([7])
ok
torch.Size([6])
2
source to target torch.Size([2, 3])
transition probabilities torch.Size([7])
ok
torch.Size([6])
2
source to target torch.Size([2, 3])
transition probabilities torch.Size([7])
ok
torch.Size([6])
2
source to target torch.Size([2, 3])
transition probabilities torch.Size([7])
ok
torch.Size([6])
2
source to target torch.Size([2, 3])
transition probabilities torch.Size([7])
ok
torch.Size([6])


/tmp/ipykernel_799/2390192856.py:18: RuntimeWarning: divide by zero encountered in log
  return x_j + np.log(transition_probability)


Likely, there is no need need to go though a convolutional layer...

In [26]:
# Calculate unique nodes and their counts
unique_nodes, counts = torch.unique(dag_graph.node_sequence, return_counts=True)
# Compute node emission probabilities
node_emission_probabilities = counts / counts.sum()

# Prepare source_to_target_edge_index_zeroth
source_to_target_edge_index_zeroth = torch.stack([
    torch.zeros_like(source_to_target_from_walks[:, 0]),
    source_to_target_from_walks[:, 0]
])

# Compute log likelihood
tot_log_lh = 0
# Calculate log likelihood for the first step
lh_l = torch.mul(torch.log(node_emission_probabilities[source_to_target_edge_index_zeroth[1]]), torch.tensor(path_counts))
tot_log_lh += lh_l.sum()

# Loop through the remaining steps
for i in range(0, l - 1):
    # Compute transition probabilities
    T = compute_transition_probabilities(m.layers[min(i + 2, cd_data.max_order)])
    # Prepare source_to_target_edge_index
    source_to_target_edge_index = source_to_target_from_walks[:, i:i + 2].T.squeeze()
    # Calculate log likelihood for current step
    lh_l = torch.mul(torch.log(T[source_to_target_edge_index[1]]), torch.tensor(path_counts))
    tot_log_lh += lh_l.sum()

# Return total log likelihood
tot_log_lh


tensor(-739.3706)

**starting from temporal network**

In [27]:
# stuff for degrees of freedom etc.
num_len_2_paths = hon_2.data.num_nodes
num_nonzero_outdegrees = len(hon_2.data.edge_index[0].unique())

In [28]:
from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index

In [29]:
tedges = [('0', '2', 1),('2', '3', 2), ('0', '2', 3), ('2', '3', 3), ('1', '2', 14), ('2', '4', 14), ('1', '2', 14),
              ('2', '4', 15)]#, ('1', '2', 5), ('2', '4', 6)]
t = pp.TemporalGraph.from_edge_list(tedges*10)

In [30]:
m = pp.MultiOrderModel.from_temporal_graph(t, max_order=2)

In [31]:
hon_1 = m.layers[1]
hon_2 = m.layers[2]
print(hon_1.data.edge_weight)
print(hon_2.data.edge_weight)

tensor([20., 20., 20., 20.])
tensor([100., 200.])


In [32]:
hon_1.data.edge_weight

tensor([20., 20., 20., 20.])

In [33]:
source_ids = hon_1.data.edge_index[0]
hon_1.data.edge_weight[source_ids]

tensor([20., 20., 20., 20.])

In [34]:
hon_1.data.edge_index[0]

tensor([0, 1, 1, 3])

In [35]:
weighted_outdegree = torch.zeros(hon_1.data.num_nodes)
weighted_outdegree = weighted_outdegree.index_add_(
    dim = 0, 
    index = hon_1.data.edge_index[0], 
    source = hon_1.data.edge_weight[source_ids]
    )
weighted_outdegree

tensor([20., 40.,  0., 20.,  0.])

In [36]:
transition_probabilities = hon_1.data.edge_weight[source_ids]/ weighted_outdegree[source_ids]
transition_probabilities

tensor([1.0000, 0.5000, 0.5000, 1.0000])

In [37]:
# this gives likelihood of all paths of lenght 2
pp.MultiOrderModel.aggregate_edge_weight(
    hon_2.data.edge_index,
    transition_probabilities,
    aggr="mul"
    )

# the we need the number of times each path has occurred

tensor([0.5000, 0.5000])

In [38]:
sort_edge_index(t.data.edge_index, t.data.t)

(tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
          3, 3, 3, 3, 3, 3, 3, 3],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4,
          4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1]]),
 tensor([ 1.,  1.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,  2.,
          3.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  2.,  3., 14., 14.,
         14., 14., 14., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 14.,
         14., 14., 14., 14., 14., 14., 14., 14., 14., 14., 14., 14., 14., 14.,
         14., 14., 14., 14., 14., 14., 14

In [39]:
node_sequence = torch.arange(t.data.num_nodes, device=edge_index.device).unsqueeze(1)
node_sequence

tensor([[0],
        [1],
        [2],
        [3],
        [4]])

In [40]:
# edge_weight = torch.ones(edge_index.size(1), device=edge_index.device)
edge_weight = g.data.edge_weight

In [41]:
l1 = pp.MultiOrderModel.aggregate_edge_index(
                edge_index=edge_index, node_sequence=node_sequence, edge_weight=edge_weight
)

IndexError: index 5 is out of bounds for dimension 0 with size 5

In [ ]:

l1.data.edge_weight

tensor([1., 1., 1., 1.])

I cannot do the model selection on the temporal graph without the path extraction. 